In [51]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_pil_image
from torchvision import datasets
from matplotlib.pyplot import imshow
from sklearn.metrics import f1_score

In [52]:
device = "cuda" if torch.cuda.is_available() else "cpu" 
print(device)

cuda


In [53]:
transform = transforms.Compose([ 
    transforms.Resize(size=(180, 180)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dir= "./data/train" # path to the train folder
validation_dir= "./data/validation" # path to the validation folder
test_dir = "./data/test" # path to test folder

train_data = datasets.ImageFolder(root=train_dir, 
                                  transform=transform) 

validation_data = datasets.ImageFolder(root=validation_dir, 
                                 transform=transform)

test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=transform)


print(f"Train data:\n{train_data}\n\nValidation data:\n{validation_data}\n\nTest data:\n{test_data}")

Train data:
Dataset ImageFolder
    Number of datapoints: 3689
    Root location: ./data/train
    StandardTransform
Transform: Compose(
               Resize(size=(180, 180), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

Validation data:
Dataset ImageFolder
    Number of datapoints: 495
    Root location: ./data/validation
    StandardTransform
Transform: Compose(
               Resize(size=(180, 180), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

Test data:
Dataset ImageFolder
    Number of datapoints: 494
    Root location: ./data/test
    StandardTransform
Transform: Compose(
               Resize(size=(180, 180), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.

In [54]:
batch_size = 4
num_workers = 2

train_set = DataLoader(dataset=train_data, 
                              batch_size=batch_size, # how many samples per batch?
                              num_workers=num_workers, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

validation_set = DataLoader(dataset=validation_data, 
                             batch_size=batch_size, 
                             num_workers=num_workers, 
                             shuffle=True) # dont usually need to shuffle validation data

test_set = DataLoader(dataset=test_data, 
                             batch_size=batch_size, 
                             num_workers=num_workers, 
                             shuffle=True) # dont usually need to shuffle testing data

train_set,validation_set,test_set

(<torch.utils.data.dataloader.DataLoader at 0x1ecc3cab280>,
 <torch.utils.data.dataloader.DataLoader at 0x1ecc3ca9720>)

In [55]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.act3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.act4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(2)

        self.fc1 = nn.Linear(256 * 11 * 11, 512)
        self.act5 = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.pool1(self.act1(self.bn1(self.conv1(x))))
        out = self.pool2(self.act2(self.bn2(self.conv2(out))))
        out = self.pool3(self.act3(self.bn3(self.conv3(out))))
        out = self.pool4(self.act4(self.bn4(self.conv4(out))))
        out = out.view(out.size(0), -1)
        out = self.act5(self.fc1(out))
        out = self.fc2(out)
        return out
    
model = SimpleCNN().to(device)

In [56]:
# I am going to add accuracies to these lists and I will use them outside of this function 
train_accuracies=[]
validation_accuracies=[]

# Function for training
def train(dataloader, model, loss_fn, optimizer, epoch):
    
    size = len(dataloader.dataset) # total number of images inside of loader
    num_batches = len(dataloader) # number of batches
    
    model.train()

    train_loss, correct = 0, 0
    

    for batch, (X, y) in enumerate(dataloader):
        # move X and y to GPU for faster training
        X, y = X.to(device), y.to(device) 

        # make prediction 
        pred = model(X)
        # calculate loss 
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward() # compute parameters gradients
        optimizer.step() # update parameters
        optimizer.zero_grad() #  reset the gradients of all parameters

        # Update training loss
        train_loss += loss.item() # item() method extracts the loss’s value as a Python float

        # Calculate training accuracy
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        if batch % 100 == 0:
            print(f'batch {batch} completed, train_loss={train_loss}')
    
    # loss and accuracy
    train_loss = train_loss / num_batches
    accuracy = 100 * correct / size
    
    # use this accuracy list for plotting accuracy with matplotlib
    train_accuracies.append(accuracy)

    # Print training accuracy and loss at the end of epoch
    print(f" Training Accuracy: {accuracy:.2f}%, Training Loss: {train_loss:.4f}")

def validation(dataloader, model, loss_fn, t):
    
    size = len(dataloader.dataset) # total number of images inside of loader
    num_batches = len(dataloader) # number of batches
    
    validation_loss, correct = 0, 0
    
    # sets the PyTorch model to evaluation mode, it will disable dropout layer
    model.eval()
    
    with torch.no_grad(): #  disable gradient calculation
        for X, y in dataloader:
            
            # move X and y to GPU for faster training
            X, y = X.to(device), y.to(device)
            pred = model(X) # make prediction
            validation_loss += loss_fn(pred, y).item() 
            
            # if prediction is correct add 1 to correct variable.
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    # loss and accuracy
    validation_loss /= num_batches
    accuracy = 100 * correct / size

    validation_accuracies.append(accuracy)

    # Print test accuracy and loss at the end of epoch
    print(f" Validation Accuracy: {accuracy:.2f}%, Validation Loss: {validation_loss:.4f}")

In [57]:
# Loss funciton, optimizer and epochs
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3)
epochs = 20

In [58]:

for t in range(epochs):
    print(f"Epoch {t+1}")
    
    train(train_set, model, loss_fn, optimizer, t)
    validation(validation_set, model, loss_fn, t)
    print("----------------------------")

print("Done!")

Epoch 1
batch 0 completed, train_loss=0.6767285466194153
batch 100 completed, train_loss=70.16140996292233
batch 200 completed, train_loss=133.44081547483802
batch 300 completed, train_loss=195.34774617478251
batch 400 completed, train_loss=255.89916813001037
batch 500 completed, train_loss=316.8931905888021
batch 600 completed, train_loss=374.76616422459483
batch 700 completed, train_loss=438.39289385452867
batch 800 completed, train_loss=499.00533770397305
batch 900 completed, train_loss=558.5208303891122
 Training Accuracy: 69.78%, Training Loss: 0.6188
 Validation Accuracy: 69.49%, Validation Loss: 0.5773
----------------------------
Epoch 2
batch 0 completed, train_loss=0.6727873086929321
batch 100 completed, train_loss=60.51677933335304
batch 200 completed, train_loss=116.26248650252819
batch 300 completed, train_loss=173.7169876843691
batch 400 completed, train_loss=233.44198049604893
batch 500 completed, train_loss=286.5778951495886
batch 600 completed, train_loss=341.613685980

## MODEL ANALYSIS

### Model testing

In [62]:
classes = ["accept", "refuse"]

dataiter = iter(test_set)
images, labels = next(dataiter)

In [63]:

# Modified validation function to compute F1 score
def validation(dataloader, model, loss_fn, t):
    size = len(dataloader.dataset) # total number of images in loader
    num_batches = len(dataloader) # number of batches
    validation_loss, correct = 0, 0
    
    # Store true and predicted labels for F1 score calculation
    all_labels = []
    all_preds = []
    
    model.eval() # Set model to evaluation mode (disables dropout)
    
    with torch.no_grad(): # Disable gradient calculation
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X) # Make prediction
            validation_loss += loss_fn(pred, y).item()
            
            # Collect predictions and labels for F1
            all_labels.extend(y.cpu().numpy())
            all_preds.extend(pred.argmax(1).cpu().numpy())
            
            # Calculate accuracy
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    # Loss and accuracy
    validation_loss /= num_batches
    accuracy = 100 * correct / size
    validation_accuracies.append(accuracy)
    
    # Calculate macro-averaged F1 score
    f1_macro = f1_score(all_labels, all_preds, average='macro')
    
    # Print validation metrics
    print(f" Validation Accuracy: {accuracy:.2f}%, Validation Loss: {validation_loss:.4f}, F1 Score (Macro): {f1_macro:.4f}")


In [64]:
print("Final evaluation on validation set:")
validation(test_set, model, loss_fn, t=epochs)

Final evaluation on validation set:
 Validation Accuracy: 98.79%, Validation Loss: 0.0492, F1 Score (Macro): 0.9855


In [65]:
BASE_PATH = "./nets/"

torch.save(model.state_dict(), "./nets/skynet5.pth")